In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import timm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained ViT model
model = timm.create_model("vit_base_patch16_224", pretrained=False, num_classes=10)
model.load_state_dict(torch.load("vit_cifar10.pth", map_location=device))
model = model.to(device)
model.eval()

# Define loss function
criterion = nn.CrossEntropyLoss()

# Data Transformations (same as before)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

# Load CIFAR-10 test dataset
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)


In [ ]:
def fgsm_attack(image, epsilon, gradient):
    """
    Generates an FGSM adversarial example by perturbing the image along the sign of the gradient.
    """
    perturbation = epsilon * gradient.sign()
    adv_image = image + perturbation
    adv_image = torch.clamp(adv_image, -1, 1)  # Keep pixel values in range
    return adv_image

def test_fgsm(model, test_loader, epsilon):
    """
    Evaluates the model on adversarial examples generated using FGSM.
    """
    correct, total = 0, 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        images.requires_grad = True

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Compute gradients
        model.zero_grad()
        loss.backward()
        gradient = images.grad.data

        # Generate adversarial example
        adv_images = fgsm_attack(images, epsilon, gradient)

        # Re-classify the perturbed image
        adv_outputs = model(adv_images)
        _, adv_pred = torch.max(adv_outputs, 1)

        # Compare predictions
        total += labels.size(0)
        correct += (adv_pred == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"FGSM Attack (ε={epsilon}) - Accuracy: {accuracy:.2f}%")
    return accuracy

# Run FGSM Attack
epsilons = [0.01, 0.1, 0.2]  # Test different strengths
for eps in epsilons:
    test_fgsm(model, test_loader, eps)


In [ ]:
def pgd_attack(image, label, model, epsilon, alpha, iters):
    """
    Generates a PGD adversarial example by iteratively perturbing the image.
    """
    perturbed_image = image.clone().detach().to(device)
    perturbed_image.requires_grad = True

    for _ in range(iters):
        outputs = model(perturbed_image)
        loss = criterion(outputs, label)

        # Compute gradients
        model.zero_grad()
        loss.backward()
        gradient = perturbed_image.grad.data

        # Apply small perturbation and project back into the allowed epsilon-ball
        perturbed_image = perturbed_image + alpha * gradient.sign()
        perturbed_image = torch.clamp(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = torch.clamp(perturbed_image, -1, 1)  # Keep values in range
        perturbed_image.requires_grad = True

    return perturbed_image

def test_pgd(model, test_loader, epsilon, alpha, iters):
    """
    Evaluates the model on adversarial examples generated using PGD.
    """
    correct, total = 0, 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Generate adversarial example using PGD
        adv_images = pgd_attack(images, labels, model, epsilon, alpha, iters)

        # Re-classify the perturbed image
        adv_outputs = model(adv_images)
        _, adv_pred = torch.max(adv_outputs, 1)

        # Compare predictions
        total += labels.size(0)
        correct += (adv_pred == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"PGD Attack (ε={epsilon}, α={alpha}, iters={iters}) - Accuracy: {accuracy:.2f}%")
    return accuracy

# Run PGD Attack
test_pgd(model, test_loader, epsilon=0.2, alpha=0.01, iters=10)


In [ ]:
import torch.nn.functional as F

def gaussian_noise_attack(image, sigma=0.1):
    """
    Adds Gaussian noise to the image.
    """
    noise = torch.randn_like(image) * sigma
    noisy_image = image + noise
    noisy_image = torch.clamp(noisy_image, -1, 1)  # Keep pixel values in range
    return noisy_image

def test_gaussian_noise(model, test_loader, sigma):
    """
    Evaluates the model on images with added Gaussian noise.
    """
    correct, total = 0, 0

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Generate noisy image
        noisy_images = gaussian_noise_attack(images, sigma)

        # Re-classify the noisy image
        outputs = model(noisy_images)
        _, predicted = torch.max(outputs, 1)

        # Compare predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Gaussian Noise (σ={sigma}) - Accuracy: {accuracy:.2f}%")
    return accuracy

# Run Gaussian Noise Attack
test_gaussian_noise(model, test_loader, sigma=0.1)
